In [1]:
import numpy as np
import pandas as pd
import math
import sklearn.preprocessing
import datetime
from TimeBasedCV import TimeBasedCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import make_scorer, r2_score
import warnings
import statsmodels.api as sm
warnings.simplefilter(action='ignore', category=Warning)
from sklearn.linear_model import HuberRegressor
# pd.set_option('display.max_rows', None)
# more

In [2]:
df = pd.read_csv('data/factors_1970.csv', parse_dates=['DATE'])
# df = pd.read_csv('data/features_subset.csv', parse_dates=['DATE'])
# df = pd.read_csv('factors_1900.csv', parse_dates=['DATE'])
# df.drop(columns=['sic2'], inplace=True)


In [3]:

#Sort observations by date and stock id
df[df.columns[2:]] = df[df.columns[2:]].astype('float32')
df = df.sort_values(by = ['DATE', 'permno'], ascending = True)
df.head()


,permno,DATE,mvel1,beta,betasq,chmom,dolvol,idiovol,indmom,mom1m,...,macro_ep,macro_bm,macro_ntis,macro_tbl,macro_tms,macro_dfy,macro_svar,macro_mkt-rf,macro_hml,macro_smb
0,10401,1970-02-27,26227356.0,0.253755,0.064391,-0.057929,13.348085,0.019852,-0.112202,-0.017995,...,2.757288,0.670147,0.035855,0.0713,-0.0126,0.0085,0.001059,5.13,3.93,-2.58
1,10604,1970-02-27,3196008.0,0.743947,0.553457,-0.228256,13.348085,0.044046,-0.252877,-0.164006,...,2.757288,0.670147,0.035855,0.0713,-0.0126,0.0085,0.001059,5.13,3.93,-2.58
2,10786,1970-02-27,1133566.5,0.608122,0.369813,-0.046833,12.456023,0.025765,-0.302840,-0.050691,...,2.757288,0.670147,0.035855,0.0713,-0.0126,0.0085,0.001059,5.13,3.93,-2.58
3,10890,1970-02-27,2662344.0,0.833271,0.694341,0.001993,13.348085,0.035138,-0.127664,-0.018268,...,2.757288,0.670147,0.035855,0.0713,-0.0126,0.0085,0.001059,5.13,3.93,-2.58
4,11260,1970-02-27,1342376.0,1.059626,1.122808,0.034205,13.348085,0.032067,-0.349402,-0.185455,...,2.757288,0.670147,0.035855,0.0713,-0.0126,0.0085,0.001059,5.13,3.93,-2.58


In [4]:
# df= df[~np.isnan(df['bm'])]
# df =df[~np.isnan(df['mvel1'])]

In [5]:
df['permno2'] = df['permno'].copy()
df['DATE2'] = df['DATE'].copy()
df = df.set_index(['DATE2','permno2'])

#Make a copy of  the "me" variable (market equity) before rank standartization to use afterwards for value weighting
df['mvel12'] = df['mvel1'].copy()

In [6]:
p=0.3 
df_top= df.groupby('DATE').apply(lambda x: x.nlargest(int(len(x)*p),'mvel1')).reset_index(drop=True)  
df_bottom = df.groupby('DATE').apply(lambda x: x.nsmallest(int(len(x)*p),'mvel1')).reset_index(drop=True)  


In [ ]:
#Standardize all independent variables
stdSc = StandardScaler()

features = df.columns[~df.columns.isin(['DATE', 'DATE2', 'mvel12', 'permno', 'permno2', 'risk_premium'])].tolist()
#Ranks in [0,1]interval
# df[features]=df.groupby('DATE')[features].rank(pct=True)
df[features] = stdSc.fit_transform(df[features].astype(float))

#Multiply by 2 and substract 1 to get ranks in interval [-1,1] 
# df[features] = 2*df[features] - 1

In [7]:
#Standardize all independent variables
features = df.columns[~df.columns.isin(['DATE', 'DATE2', "mvel2",'permno',"permno2",'risk_premium'])].tolist()

# #Ranks in [0,1]interval
df[features]=df.groupby('DATE')[features].rank(pct=True)

# #Multiply by 2 and substract 1 to get ranks in interval [-1,1] 
df[features] = 2*df[features] - 1

In [8]:
def calculate_r2(y_true, y_pred, in_sample=True, benchmark=None):
    if in_sample:
        return 1 - (np.sum((y_true - y_pred) ** 2) / 
                    np.sum((y_true - np.mean(y_true)) ** 2))
    else:
        if benchmark is None:
            raise ValueError("Benchmark must be provided for out-of-sample R-squared calculation.")
        return 1 - (np.sum((y_true - y_pred) ** 2) / 
                    np.sum((y_true - benchmark) ** 2))

In [23]:
tscv = TimeBasedCV(train_period=60,
                   val_period=24,
                #    test_period=12,
                   freq='months')

features = df.columns[~df.columns.isin(['permno', 'permno2', 'mvel12', 'DATE2', 'risk_premium'])].tolist()

X = df[features]
y = df[['risk_premium']]

X_train = sm.add_constant(X)

predictions = []
y_val_list = []
dates = []
r2_train_list = []
r2_val_list = []
dic_r2_all = {}

for train_index, val_index in tscv.split(X, first_split_date= datetime.date(1985,1,31), second_split_date= datetime.date(1987,1,31)):

    X_train   = X.loc[train_index].drop('DATE', axis=1)
    y_train = y.loc[train_index]

    X_val   = X.loc[val_index].drop('DATE', axis=1)
    y_val = y.loc[val_index]
    lm = LinearRegression()
   
    lm.fit(X_train, y_train)
    preds = lm.predict(X_val) 
    #Save predictions, dates and the true values of the dependent variable to list  
    predictions.append(preds)
    dates.append(y_val.index)
    y_val_list.append(y_val)
    
    #Calculate OOS model performance the for current window
    r2 = 1-np.sum(pow(y_val-preds,2))/np.sum(pow(y_val,2))
    #Save OOS model performance and the respective month to dictionary
    dic_r2_all["r2." + str(y_val.index)] = r2

predictions_all_full = np.concatenate(predictions, axis=0)
y_test_list_all_full = np.concatenate(y_val_list, axis=0)
dates_all_full = np.concatenate(dates, axis=0)
R2FULL = 1-np.sum(pow(y_test_list_all_full-predictions_all_full,2))/np.sum(pow(y_test_list_all_full,2))
print("R2OOS Linear Regression: ", R2FULL)


Train period: 1980-01-31 - 1985-01-31 ,val period: 1985-01-31 - 1987-01-31 # train records 18589 ,# val records 11286
Train period: 1981-01-31 - 1986-01-31 ,val period: 1986-01-31 - 1988-01-31 # train records 20125 ,# val records 13057
Train period: 1982-01-31 - 1987-01-31 ,val period: 1987-01-31 - 1989-01-31 # train records 22963 ,# val records 12572
Train period: 1983-01-31 - 1988-01-31 ,val period: 1988-01-31 - 1990-01-31 # train records 26711 ,# val records 12781
Train period: 1984-01-31 - 1989-01-31 ,val period: 1989-01-31 - 1991-01-31 # train records 28226 ,# val records 13403
Train period: 1985-01-31 - 1990-01-31 ,val period: 1990-01-31 - 1992-01-31 # train records 30708 ,# val records 13616
Train period: 1986-01-31 - 1991-01-31 ,val period: 1991-01-31 - 1993-01-31 # train records 32391 ,# val records 15806
Train period: 1987-01-31 - 1992-01-31 ,val period: 1992-01-31 - 1994-01-31 # train records 33038 ,# val records 17371
Train period: 1988-01-31 - 1993-01-31 ,val period: 1993-

In [24]:
tscv = TimeBasedCV(train_period=60,
                   val_period=24,
                   freq='months')

features = df.columns[~df.columns.isin(['permno', 'permno2', 'mvel12', 'DATE2', 'risk_premium'])].tolist()

X = df_top[features]
y = df_top[['risk_premium']]

X_train = sm.add_constant(X)

predictions_top = []
y_test_list_top =[]
dates_top = []
dic_r2_all_top = {}

for train_index, val_index in tscv.split(X, first_split_date= datetime.date(1985,1,31), second_split_date= datetime.date(1987,1,31)):

    X_train   = X.loc[train_index].drop('DATE', axis=1)
    y_train = y.loc[train_index]

    X_val   = X.loc[val_index].drop('DATE', axis=1)
    y_val = y.loc[val_index]
    lm = LinearRegression()
    lm.fit(X_train, y_train)

    preds = lm.predict(X_val)    
    predictions_top.append(preds)
    dates_top.append(y_val.index)
    y_test_list_top.append(y_val)

    r2 = 1-np.sum(pow(y_val-preds,2))/np.sum(pow(y_val,2))
    dic_r2_all_top["r2." + str(y_val.index)] = r2

predictions_all_top= np.concatenate(predictions_top, axis=0)
y_test_list_all_top= np.concatenate(y_test_list_top, axis=0) 
dates_all_top= np.concatenate(dates_top, axis=0)

#Calculate OOS model performance over the entire test period in line with Gu et al (2020)
R2TOP = 1-np.sum(pow(y_test_list_all_top-predictions_all_top,2))/np.sum(pow(y_test_list_all_top,2))
print("R2OOS Linear Regression: ", R2TOP)


Train period: 1980-01-31 - 1985-01-31 ,val period: 1985-01-31 - 1987-01-31 # train records 5549 ,# val records 3377
Train period: 1981-01-31 - 1986-01-31 ,val period: 1986-01-31 - 1988-01-31 # train records 6012 ,# val records 3906
Train period: 1982-01-31 - 1987-01-31 ,val period: 1987-01-31 - 1989-01-31 # train records 6863 ,# val records 3764
Train period: 1983-01-31 - 1988-01-31 ,val period: 1988-01-31 - 1990-01-31 # train records 7988 ,# val records 3826
Train period: 1984-01-31 - 1989-01-31 ,val period: 1989-01-31 - 1991-01-31 # train records 8445 ,# val records 4008
Train period: 1985-01-31 - 1990-01-31 ,val period: 1990-01-31 - 1992-01-31 # train records 9190 ,# val records 4072
Train period: 1986-01-31 - 1991-01-31 ,val period: 1991-01-31 - 1993-01-31 # train records 9691 ,# val records 4732
Train period: 1987-01-31 - 1992-01-31 ,val period: 1992-01-31 - 1994-01-31 # train records 9885 ,# val records 5202
Train period: 1988-01-31 - 1993-01-31 ,val period: 1993-01-31 - 1995-01-

In [25]:
tscv = TimeBasedCV(train_period=60,
                   val_period=24,
                   freq='months')


features = df.columns[~df.columns.isin(['permno', 'permno2', 'mvel12', 'DATE2', 'risk_premium'])].tolist()
X = df_bottom[features]
y = df_bottom["risk_premium"]

predictions_bottom = []
y_test_list_bottom =[]
dates_bottom = []
dic_r2_all_bottom = {}


for train_index, val_index in tscv.split(X, first_split_date= datetime.date(1985,1,31), second_split_date= datetime.date(1987,1,31)):

    X_train   = X.loc[train_index].drop('DATE', axis=1)
    y_train = y.loc[train_index]

    X_val   = X.loc[val_index].drop('DATE', axis=1)
    y_val = y.loc[val_index]
    lm = LinearRegression()
    lm.fit(X_train, y_train)
   
    preds = lm.predict(X_val)    
    predictions_bottom.append(preds)
    dates_bottom.append(y_val.index)
    y_test_list_bottom.append(y_val)

    r2 = 1-np.sum(pow(y_val-preds,2))/np.sum(pow(y_val,2))
    dic_r2_all_bottom["r2." + str(y_val.index)] = r2

predictions_all_bottom= np.concatenate(predictions_bottom, axis=0)
y_test_list_all_bottom= np.concatenate(y_test_list_bottom, axis=0) 
dates_all_bottom= np.concatenate(dates_bottom, axis=0)

#Calculate OOS model performance over the entire test period in line with Gu et al (2020)
R2BOTTOM = 1-np.sum(pow(y_test_list_all_bottom-predictions_all_bottom,2))/np.sum(pow(y_test_list_all_bottom,2))
print("R2OOS Linear Regression: ", R2BOTTOM)


Train period: 1980-01-31 - 1985-01-31 ,val period: 1985-01-31 - 1987-01-31 # train records 5549 ,# val records 3377
Train period: 1981-01-31 - 1986-01-31 ,val period: 1986-01-31 - 1988-01-31 # train records 6012 ,# val records 3906
Train period: 1982-01-31 - 1987-01-31 ,val period: 1987-01-31 - 1989-01-31 # train records 6863 ,# val records 3764
Train period: 1983-01-31 - 1988-01-31 ,val period: 1988-01-31 - 1990-01-31 # train records 7988 ,# val records 3826
Train period: 1984-01-31 - 1989-01-31 ,val period: 1989-01-31 - 1991-01-31 # train records 8445 ,# val records 4008
Train period: 1985-01-31 - 1990-01-31 ,val period: 1990-01-31 - 1992-01-31 # train records 9190 ,# val records 4072
Train period: 1986-01-31 - 1991-01-31 ,val period: 1991-01-31 - 1993-01-31 # train records 9691 ,# val records 4732
Train period: 1987-01-31 - 1992-01-31 ,val period: 1992-01-31 - 1994-01-31 # train records 9885 ,# val records 5202
Train period: 1988-01-31 - 1993-01-31 ,val period: 1993-01-31 - 1995-01-

In [26]:
chart = np.array([[R2FULL],
                  [R2TOP],
                  [R2BOTTOM]])

r2_lm = pd.DataFrame(chart, columns=['Linear Regression'],
                     index=['Full Sample', 'Large Firms', 'Small Firms'])

r2_lm

,Linear Regression
Full Sample,0.006685
Large Firms,0.193474
Small Firms,0.197597


In [27]:
r2_lm.to_csv(r'r2_linear_model2.csv')

In [ ]:
features = df.columns[~df.columns.isin(['permno','permno','DATE','DATE2','mvel12','risk_premium', 'year'])].tolist()
df['YEAR'] = df['DATE'].dt.year

X_train = df[features].loc[(df['YEAR']>=2012) & (df['YEAR']<=2016)]
y_train = df['risk_premium'].loc[(df['YEAR']>=2012) & (df['YEAR']<=2016)]

X_val = df[features].loc[(df['YEAR']>=2017) & (df['YEAR']<=2018)]
y_val = df['risk_premium'].loc[(df['YEAR']>=2017) & (df['YEAR']<=2018)]


lm_model = LinearRegression()
lm_model.fit(X_train, y_train)
y_pred_train = lm_model.predict(X_train) 

y_pred_val = lm_model.predict(X_val) 

r2_score_train = r2_score(y_train, y_pred_train)
r2_score_val = r2_score(y_val, y_pred_val)

print(f'R2 score on training set: {r2_score_train}')
print(f'R2 score on validation set: {r2_score_val}')

In [ ]:
for j in features:
    globals()['df_' + str(j)] =  df.copy()
    globals()['df_' + str(j)][str(j)] = 0

In [ ]:
dic = {}

    
for j in features:
    df_var = globals()['df_' + str(j)]
    
    X_train = df_var[features].loc[(df_var["year"]>=2012) & (df_var["year"]<=2016)]
    y_train = df_var['risk_premium'].loc[(df_var["year"]>=2012) & (df_var["year"]<=2016)]

    X_val = df_var[features].loc[(df_var["year"]>=2017) & (df_var["year"]<=2018)]
    y_val = df_var['risk_premium'].loc[(df_var["year"]>=2017) & (df_var["year"]<=2018)]
    
    lm_model = LinearRegression()
    lm_model.fit(X_train, y_train)

    y_pred_train = lm_model.predict(X_train) 

    y_pred_val = lm_model.predict(X_val) 

    r2_score_train = r2_score(y_train, y_pred_train)
    r2_score_val = r2_score(y_val, y_pred_val)



In [28]:
yhat = predictions_all_full.tolist()
y_true = y_test_list_all_full.tolist()
i = dates_all_full.tolist()

results = pd.DataFrame(
    {'identifier': i,
     'yhat': yhat,
     'y_true': y_true
    })


results["identifier"]= results["identifier"].astype("str")
results["date"] = results["identifier"].str[12:22]
results["id"] = results["identifier"].str[35:40]
results.drop(["identifier"],axis = 1, inplace=True)
results['date'] = pd.to_datetime(results['date'], format='%Y-%m-%d')
results['MonthYear'] = results['date'].dt.to_period('M')
results = results.sort_values(by = ['date', 'id'], ascending = True)
results = results.set_index(['MonthYear','id'])
results.head()


yhat                y_true       date
MonthYear id                                                           
1985-01   10137   [-9.950548941563085]  [-6.745699882507324] 1985-01-31
          10145  [-10.669148495122052]   [4.993500232696533] 1985-01-31
          10161   [-9.965771155553421]   [8.103799819946289] 1985-01-31
          10225  [-10.381458743619724]  [-8.828200340270996] 1985-01-31
          10233   [-9.940727314489116]  [-6.663899898529053] 1985-01-31

In [ ]:
results['yhat'] = results['yhat'].apply(lambda x: x[0])
results['y_true'] = results['y_true'].apply(lambda x: x[0])
results.to_csv('results.csv', index=False)

In [32]:
data = df[['mvel12', 'macro_tbl', 'macro_svar']].copy()
data.reset_index(inplace=True)
data['permno2'] = data['permno2'].astype('str')
data['MonthYear'] = data['DATE2'].dt.to_period('M')
data.drop('DATE2', axis=1, inplace=True)
data.rename(columns={'permno2': 'id'}, inplace=True)
data.rename(columns={'mvel12': 'market_cap'}, inplace=True)
data.rename(columns={'macro_tbl': 'risk_free_rate'}, inplace=True)
data = data.set_index(['MonthYear','id'])


In [33]:
bigdata = pd.merge(results, data,left_index=True, right_index=True)
bigdata.reset_index(inplace=True)
bigdata['returns'] = bigdata['y_true'] + bigdata['risk_free_rate']
bigdata['MonthYear1'] = bigdata['MonthYear'].copy()
bigdata['MonthYear'] = bigdata['MonthYear'].astype('int64')
bigdata['NumMonth'] = bigdata['MonthYear'] - 251
bigdata['NumMonth'].unique()

,MonthYear,id,yhat,y_true,date,market_cap,risk_free_rate,macro_svar,returns,MonthYear1,NumMonth
0,180,10137,-9.950549,-6.7457,1985-01-31,-0.412073,0.002625,0.002625,-6.743075,1985-01,-71
1,180,10145,-10.669148,4.9935,1985-01-31,0.370079,0.002625,0.002625,4.996125,1985-01,-71
2,180,10161,-9.965771,8.1038,1985-01-31,-0.769029,0.002625,0.002625,8.106424,1985-01,-71
3,180,10225,-10.381459,-8.8282,1985-01-31,0.543307,0.002625,0.002625,-8.825576,1985-01,-71
4,180,10233,-9.940727,-6.6639,1985-01-31,-0.049869,0.002625,0.002625,-6.661275,1985-01,-71
...,...,...,...,...,...,...,...,...,...,...,...
1190349,612,93393,-4.337438,-5.4710,2021-01-29,-0.964940,0.000398,0.000398,-5.470602,2021-01,361
1190350,612,93419,-4.893715,-2.4747,2021-01-29,-0.098805,0.000398,0.000398,-2.474302,2021-01,361
1190351,612,93423,-5.191129,0.2333,2021-01-29,-0.258167,0.000398,0.000398,0.233698,2021-01,361
1190352,612,93427,-3.746458,1.6799,2021-01-29,-0.267729,0.000398,0.000398,1.680298,2021-01,361


In [34]:
for i in bigdata['NumMonth'].unique():
    globals()['df_' + str(i)] = bigdata[bigdata['NumMonth'] == i]

for i in bigdata["NumMonth"].unique():
    globals()['df_' + str(i)]["rank"]= globals()['df_' + str(i)]['yhat'].rank(method='first')
    
for i in bigdata["NumMonth"].unique():
    globals()['df_' + str(i)]["DecileRank"]=pd.qcut(globals()['df_' + str(i)]['rank'].values, 10, labels = False)

#Drop normal rank, retain only decile ranks 
for i in bigdata["NumMonth"].unique():
     globals()['df_' + str(i)].drop('rank', axis=1, inplace=True)

In [35]:

for i in bigdata["NumMonth"].unique():
    for j,g in globals()['df_' + str(i)].groupby('DecileRank'):
        globals()['df_' + str(i)+ "_" + str(j)] =  g

for j in np.arange(0,10,1):
    globals()['rank_' + str(j)] = pd.concat([globals()['df_1_'+ str(j)], globals()['df_2_'+ str(j)]], axis=0)
    
# Generate 10 Dataframes for the 10 Decile portfolios 0-9: rank_9: top portfolio, rank_0: bottom portfolio
for i in np.arange(2,361,1):
    for j in np.arange(0,10,1):
        globals()['rank_' + str(j)] = pd.concat([globals()['rank_' + str(j)], globals()['df_' + str(i+1)+ "_" + str(j)]], axis = 0)

In [36]:
rank_9

,MonthYear,id,yhat,y_true,date,market_cap,risk_free_rate,macro_svar,returns,MonthYear1,NumMonth,DecileRank
69652,252,10078,-3.243496,26.5585,1991-01-31,-0.108108,0.001931,0.001931,26.560431,1991-01,1,9
69653,252,10078,-2.174278,26.5585,1991-01-31,-0.108108,0.001931,0.001931,26.560431,1991-01,1,9
69655,252,10104,-3.856875,8.0957,1991-01-31,-0.934363,0.001931,0.001931,8.097631,1991-01,1,9
69670,252,10401,-4.222955,2.1088,1991-01-31,0.972973,0.001931,0.001931,2.110730,1991-01,1,9
69671,252,10401,-3.613610,2.1088,1991-01-31,0.972973,0.001931,0.001931,2.110730,1991-01,1,9
...,...,...,...,...,...,...,...,...,...,...,...,...
1190310,612,93096,-2.961586,-7.5208,2021-01-29,0.876494,0.000398,0.000398,-7.520402,2021-01,361,9
1190315,612,93143,-3.594337,-2.8690,2021-01-29,0.196813,0.000398,0.000398,-2.868602,2021-01,361,9
1190319,612,93147,-3.582689,-0.0194,2021-01-29,0.424701,0.000398,0.000398,-0.019002,2021-01,361,9
1190325,612,93189,-3.282593,-0.2801,2021-01-29,0.047809,0.000398,0.000398,-0.279702,2021-01,361,9


In [ ]:
rank_0['res'] = abs(rank_0['y_true'] - rank_0['yhat'])

In [ ]:
sum(rank_0.groupby('MonthYear')['yhat'].mean()/rank_9.groupby('MonthYear')['returns'].std())

In [ ]:
np.mean(rank_9.groupby('NumMonth')['yhat'].mean())

In [ ]:
np.mean(rank_9.groupby('MonthYear')['y_true'].mean())

In [37]:

# Get equal und value weights per stock per month in each decile portfolio:
# ew = Equally weighted
# vw = Value weighted

for j in np.arange(0,10,1):
    globals()['rank_' + str(j)]["eq_weights"] = 1/globals()['rank_' + str(j)].groupby('MonthYear')["id"].transform('size')
    globals()['rank_' + str(j)]["me_weights"] = globals()['rank_' + str(j)]["market_cap"]/globals()['rank_' + str(j)].groupby('MonthYear')["market_cap"].transform('sum')

In [38]:

# Weighted excess return per stock in t+1
for j in np.arange(0,10,1):
    globals()['rank_' + str(j)]['excess_return_stock_ew'] = globals()['rank_' + str(j)]["y_true"]*globals()['rank_' + str(j)]["eq_weights"]
    globals()['rank_' + str(j)]['excess_return_stock_vw'] = globals()['rank_' + str(j)]["y_true"]*globals()['rank_' + str(j)]["me_weights"]

In [39]:

# weighted return per stock in t+1 (to use for the sharpe ratio)
for j in np.arange(0,10,1):
    globals()['rank_' + str(j)]['return_stock_ew'] = globals()['rank_' + str(j)]["returns"]*globals()['rank_' + str(j)]["eq_weights"]
    globals()['rank_' + str(j)]['return_stock_vw'] = globals()['rank_' + str(j)]["returns"]*globals()['rank_' + str(j)]["me_weights"]

In [40]:

# Portfolio excess return in t+1
for j in np.arange(0,10,1):
    globals()['rank_' + str(j)]['excess_return_portfolio_ew'] = globals()['rank_' + str(j)].groupby('MonthYear')["excess_return_stock_ew"].transform('sum')
    globals()['rank_' + str(j)]['excess_return_portfolio_vw'] = globals()['rank_' + str(j)].groupby('MonthYear')["excess_return_stock_vw"].transform('sum')

In [41]:
# Portfolio return in t+1 (to use for the sharpe ratio) 
for j in np.arange(0,10,1):
    globals()['rank_' + str(j)]['return_portfolio_ew'] = globals()['rank_' + str(j)].groupby('MonthYear')["return_stock_ew"].transform('sum')
    globals()['rank_' + str(j)]['return_portfolio_vw'] = globals()['rank_' + str(j)].groupby('MonthYear')["return_stock_vw"].transform('sum')

In [42]:

# Weighted predicted excess return per stock in t+1
for j in np.arange(0,10,1):
    globals()['rank_' + str(j)]['pred_excess_return_stock_ew'] = globals()['rank_' + str(j)]["yhat"]*globals()['rank_' + str(j)]["eq_weights"]
    globals()['rank_' + str(j)]['pred_excess_return_stock_vw'] = globals()['rank_' + str(j)]["yhat"]*globals()['rank_' + str(j)]["me_weights"]

In [43]:
# Portfolio predicted excess return in t+1
for j in np.arange(0,10,1):
    globals()['rank_' + str(j)]['pred_excess_return_portfolio_ew'] = globals()['rank_' + str(j)].groupby('MonthYear')["pred_excess_return_stock_ew"].transform('sum')
    globals()['rank_' + str(j)]['pred_excess_return_portfolio_vw'] = globals()['rank_' + str(j)].groupby('MonthYear')["pred_excess_return_stock_vw"].transform('sum')

In [44]:
# Generate dataframes, containing the portfolio returns on mohtly basis for each decile portfolio
# e.g., montly_rank_0: dataframe, containing only the monthly portfolio excess returns (predicted and real) 
# for  the bottom rank

for j in np.arange(0,10,1):
    globals()['montly_rank_' + str(j)] = globals()['rank_' + str(j)][["MonthYear1", "DecileRank",
                                                                      "excess_return_portfolio_ew",
                                                                      "excess_return_portfolio_vw", 
                                                                      "pred_excess_return_portfolio_ew",
                                                                      "pred_excess_return_portfolio_vw",
                                                                      "return_portfolio_ew",
                                                                      "return_portfolio_vw"]]
    
for j in np.arange(0,10,1):
    globals()['montly_rank_' + str(j)]=globals()['montly_rank_' + str(j)].drop_duplicates()
    globals()['montly_rank_' + str(j)]=globals()['montly_rank_' + str(j)].set_index("MonthYear1")

In [45]:

for j in np.arange(0,10,1):
    #Time-series average of realized excess returns
    globals()["ew_mean_return_rank_" +  str(j)]= globals()['montly_rank_' + str(j)]["excess_return_portfolio_ew"].mean()
    globals()["vw_mean_return_rank_" +  str(j)]= globals()['montly_rank_' + str(j)]["excess_return_portfolio_vw"].mean()
    #Time-series average of predicted excess returns
    globals()["ew_mean_pred_return_rank_" +  str(j)]= globals()['montly_rank_' + str(j)]["pred_excess_return_portfolio_ew"].mean()
    globals()["vw_mean_pred_return_rank_" +  str(j)]= globals()['montly_rank_' + str(j)]["pred_excess_return_portfolio_vw"].mean()
    #Standard deviation of realized excess returns
    globals()["std_ew_rank_" +  str(j)]= globals()['montly_rank_' + str(j)]["excess_return_portfolio_ew"].std()
    globals()["std_vw_rank_" +  str(j)]= globals()['montly_rank_' + str(j)]["excess_return_portfolio_vw"].std()
    #Annualized sharpe ratio of realized excess returns
    globals()["sharpe_ew_rank_" +  str(j)]= (globals()['montly_rank_' + str(j)]["excess_return_portfolio_ew"].mean()/globals()['montly_rank_' + str(j)]["return_portfolio_ew"].std())* np.sqrt(12)
    globals()["sharpe_vw_rank_" +  str(j)]= (globals()['montly_rank_' + str(j)]["excess_return_portfolio_vw"].mean()/globals()['montly_rank_' + str(j)]["return_portfolio_vw"].std())* np.sqrt(12)

In [46]:
# For the zero-net-investment long-short portfolio the top (long) and bottom(short) decile portfolios are needed

long_monthly = rank_9[["NumMonth","MonthYear1", "DecileRank", "excess_return_portfolio_ew",
                       "excess_return_portfolio_vw","pred_excess_return_portfolio_ew",
                       "pred_excess_return_portfolio_vw","return_portfolio_ew",
                        "return_portfolio_vw"]].drop_duplicates()

short_monthly = rank_0[["NumMonth","MonthYear1", "DecileRank", "excess_return_portfolio_ew",
                       "excess_return_portfolio_vw","pred_excess_return_portfolio_ew",
                       "pred_excess_return_portfolio_vw","return_portfolio_ew",
                        "return_portfolio_vw"]].drop_duplicates()

# Create a column, indication the stategy 
long_monthly["Strategy"]= "long"
short_monthly["Strategy"]= "short"

# Merge to get the zero net investment portfolio
zeronet_monthly= pd.concat([long_monthly, short_monthly])
zeronet_monthly = zeronet_monthly.sort_values(by = ['NumMonth',"Strategy"])
zeronet_monthly["return_portfolio_vw"] = zeronet_monthly["return_portfolio_vw"].astype('float64')

#Create two new columns containing the exess return of the portfolio and initially set the values to zero.
zeronet_monthly["excess_return_zeronet_ew"] =0
zeronet_monthly["excess_return_zeronet_vw"] =0

# excess return zeronet in t = (weigted excess return long in t) - (weigted excess return short in t)
for i in range(0, len(zeronet_monthly)):
    if zeronet_monthly.iloc[i,9] == "long":
        zeronet_monthly.iloc[i, -2] = zeronet_monthly.iloc[i, 3]-zeronet_monthly.iloc[i+1, 3]
    else:
        zeronet_monthly.iloc[i, -2] = zeronet_monthly.iloc[i-1, 3]-zeronet_monthly.iloc[i, 3]
        
for i in range(0, len(zeronet_monthly)):
    if zeronet_monthly.iloc[i,9] == "long":
        zeronet_monthly.iloc[i, -1] = zeronet_monthly.iloc[i, 4]-zeronet_monthly.iloc[i+1, 4]
    else:
        zeronet_monthly.iloc[i, -1] = zeronet_monthly.iloc[i-1, 4]-zeronet_monthly.iloc[i, 4]

#Create two new columns containing predicted the exess return of the portfolio and initially set the values to zero.
zeronet_monthly["pred_excess_return_zeronet_ew"] =0
zeronet_monthly["pred_excess_return_zeronet_vw"] =0

# predicted excess return zeronet in t = (weigted predicted excess return long in t) - (weigted predicted excess return short in t)
for i in range(0, len(zeronet_monthly)):
    if zeronet_monthly.iloc[i,9] == "long":
        zeronet_monthly.iloc[i, -2] = zeronet_monthly.iloc[i, 5]-zeronet_monthly.iloc[i+1, 5]
    else:
        zeronet_monthly.iloc[i, -2] = zeronet_monthly.iloc[i-1, 5]-zeronet_monthly.iloc[i, 5]
        
for i in range(0, len(zeronet_monthly)):
    if zeronet_monthly.iloc[i,9] == "long":
        zeronet_monthly.iloc[i, -1] = zeronet_monthly.iloc[i, 6]-zeronet_monthly.iloc[i+1, 6]
    else:
        zeronet_monthly.iloc[i, -1] = zeronet_monthly.iloc[i-1, 6]-zeronet_monthly.iloc[i, 6]

#Create two new columns containing return of the portfolio and initially set the values to zero.       
zeronet_monthly["return_zeronet_ew"] =0
zeronet_monthly["return_zeronet_vw"] =0

# return zeronet in t = (weigted return long in t) - (weigted return short in t)
for i in range(0, len(zeronet_monthly)):
    if zeronet_monthly.iloc[i,9] == "long":
        zeronet_monthly.iloc[i, -2] = zeronet_monthly.iloc[i, 7]-zeronet_monthly.iloc[i+1, 7]
    else:
        zeronet_monthly.iloc[i, -2] = zeronet_monthly.iloc[i-1, 7]-zeronet_monthly.iloc[i, 7]
        
for i in range(0, len(zeronet_monthly)):
    if zeronet_monthly.iloc[i,9] == "long":
        zeronet_monthly.iloc[i, -1] = zeronet_monthly.iloc[i, 8]-zeronet_monthly.iloc[i+1, 8]
    else:
        zeronet_monthly.iloc[i, -1] = zeronet_monthly.iloc[i-1, 8]-zeronet_monthly.iloc[i, 8]

In [47]:
zeronet_monthly

,NumMonth,MonthYear1,DecileRank,excess_return_portfolio_ew,excess_return_portfolio_vw,pred_excess_return_portfolio_ew,pred_excess_return_portfolio_vw,return_portfolio_ew,return_portfolio_vw,Strategy,excess_return_zeronet_ew,excess_return_zeronet_vw,pred_excess_return_zeronet_ew,pred_excess_return_zeronet_vw,return_zeronet_ew,return_zeronet_vw
69652,1,1991-01,9,-0.006890,-5.519047,-3.518273,-2.800851,-0.004960,-5.517116,long,-1.186116,-17.952754,3.411566,2.876860,-1.186116,-17.952754
69656,1,1991-01,0,1.179226,12.433707,-6.929839,-5.677711,1.181156,12.435638,short,-1.186116,-17.952754,3.411566,2.876860,-1.186116,-17.952754
70688,2,1991-02,9,1.867925,-4.449904,-4.565645,-4.158577,1.869767,-4.448062,long,-0.171483,-48.660076,3.298111,2.798375,-0.171483,-48.660076
70692,2,1991-02,0,2.039408,44.210172,-7.863757,-6.956952,2.041250,44.212013,short,-0.171483,-48.660076,3.298111,2.798375,-0.171483,-48.660076
71774,3,1991-03,9,-3.305028,-10.384236,-4.917770,-3.635232,-3.303277,-10.382484,long,-0.436824,-25.264882,4.372078,3.992753,-0.436824,-25.264882
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183209,359,2020-11,0,19.663007,18.743534,-2.288670,-2.206594,19.663448,18.743975,short,-9.122067,-13.718118,2.372804,2.304532,-9.122067,-13.718118
1185431,360,2020-12,9,5.297752,0.150644,-1.962180,-2.041428,5.298166,0.151058,long,-1.088729,-6.175783,2.472344,2.327706,-1.088729,-6.175783
1185493,360,2020-12,0,6.386481,6.326427,-4.434524,-4.369134,6.386895,6.326842,short,-1.088729,-6.175783,2.472344,2.327706,-1.088729,-6.175783
1187844,361,2021-01,9,0.050683,-8.161272,-3.262973,-3.287607,0.051082,-8.160874,long,-1.402181,-10.463185,2.374793,2.292315,-1.402181,-10.463185


In [48]:
#Only the measures at portfolio level are needed
zeronet_monthly = zeronet_monthly[['NumMonth', 'MonthYear1', 'excess_return_zeronet_ew',
                                   'excess_return_zeronet_vw', 'pred_excess_return_zeronet_ew',
                                   'pred_excess_return_zeronet_vw','return_zeronet_ew',
                                   'return_zeronet_vw']].drop_duplicates()

In [49]:

#Calculate zero-net portfolio performance measures                                            
#Time-series average of realized excess returns                                             
ew_mean_return_zeronet= zeronet_monthly["excess_return_zeronet_ew"].mean()
vw_mean_return_zeronet= zeronet_monthly["excess_return_zeronet_vw"].mean()
#Time-series average of predicted excess returns
ew_mean_pred_return_zeronet = zeronet_monthly["pred_excess_return_zeronet_ew"].mean()
vw_mean_pred_return_zeronet = zeronet_monthly["pred_excess_return_zeronet_vw"].mean()
#Standard deviation of realized excess returns
std_ew_zeronet = zeronet_monthly["excess_return_zeronet_ew"].std()
std_vw_zeronet = zeronet_monthly["excess_return_zeronet_vw"].std()
#Annualized sharpe ratio of realized excess returns
sharpe_ew_zeronet = (zeronet_monthly["excess_return_zeronet_ew"].mean()/zeronet_monthly["return_zeronet_ew"].std())* np.sqrt(12)
sharpe_vw_zeronet = (zeronet_monthly["excess_return_zeronet_vw"].mean()/zeronet_monthly["return_zeronet_vw"].std())* np.sqrt(12)

In [50]:
chart_np = np.array([[ew_mean_pred_return_rank_0, ew_mean_return_rank_0, std_ew_rank_0, sharpe_ew_rank_0],
                     [ew_mean_pred_return_rank_1, ew_mean_return_rank_1, std_ew_rank_1, sharpe_ew_rank_1],
                     [ew_mean_pred_return_rank_2, ew_mean_return_rank_2, std_ew_rank_2, sharpe_ew_rank_2],
                     [ew_mean_pred_return_rank_3, ew_mean_return_rank_3, std_ew_rank_3, sharpe_ew_rank_3],
                     [ew_mean_pred_return_rank_4, ew_mean_return_rank_4, std_ew_rank_4, sharpe_ew_rank_4],
                     [ew_mean_pred_return_rank_5, ew_mean_return_rank_5, std_ew_rank_5, sharpe_ew_rank_5],
                     [ew_mean_pred_return_rank_6, ew_mean_return_rank_6, std_ew_rank_6, sharpe_ew_rank_6],
                     [ew_mean_pred_return_rank_7, ew_mean_return_rank_7, std_ew_rank_7, sharpe_ew_rank_7],
                     [ew_mean_pred_return_rank_8, ew_mean_return_rank_8, std_ew_rank_8, sharpe_ew_rank_8],
                     [ew_mean_pred_return_rank_9, ew_mean_return_rank_9, std_ew_rank_9, sharpe_ew_rank_9],
                     [ew_mean_pred_return_zeronet, ew_mean_return_zeronet, std_ew_zeronet, sharpe_ew_zeronet]])

ew_df = pd.DataFrame(chart_np, columns=['Pred', 'Real', 'Std', 'Sharpe'],
                              index=['Low (L)', '2', '3', '4', '5','6','7','8',"9",'High (H)', "H-L"])

ew_df['Pred'] = pd.Series(["{0:.2f}%".format(val) for val in ew_df['Pred']], index = ew_df.index)
ew_df['Real'] = pd.Series(["{0:.2f}%".format(val) for val in ew_df['Real']], index = ew_df.index)
ew_df['Std'] = pd.Series(["{0:.2f}%".format(val) for val in ew_df['Std']], index = ew_df.index)
ew_df['Sharpe'] = pd.Series([("%.2f" % round(val, 2)) for val in ew_df['Sharpe']], index = ew_df.index)
ew_df

,Pred,Real,Std,Sharpe
Low (L),-5.17%,-2.01%,6.11%,-1.14
2,-4.45%,-1.73%,5.25%,-1.14
3,-4.09%,-1.65%,5.07%,-1.13
4,-3.78%,-1.61%,4.94%,-1.13
5,-3.45%,-1.63%,4.88%,-1.16
6,-2.91%,-1.66%,5.01%,-1.15
7,-2.59%,-1.62%,4.94%,-1.13
8,-2.29%,-1.58%,5.04%,-1.09
9,-1.94%,-1.61%,5.02%,-1.11
High (H),-1.27%,-1.56%,5.55%,-0.97
